# Homework

In [ ]:
import os
import requests
import threading
import subprocess

import mlflow
from mlflow.tracking import MlflowClient

## Q1. Install MLflow

Run the command `mlflow --version` and check the output. What's the version that you have?

In [2]:
print(mlflow.__version__)

2.13.0


## Q2. Download and preprocess the data

Download the data (**Green** Taxi Trip Records) for January, February, and March 2023 ([NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page))

In [3]:
def download_file(url, directory):
    response = requests.get(url)
    filename = os.path.join(directory, url.split("/")[-1])
    with open(filename, 'wb') as f:
        f.write(response.content)
    return filename

In [4]:
DATA_FOLDER = "./data"

datasets = [
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet"
]

for dataset in datasets:
    filename = os.path.join(DATA_FOLDER, dataset.split("/")[-1])
    if not os.path.exists(filename):
        download_file(dataset, DATA_FOLDER)

Use the script `preprocess_data.py` to preprocess the data. The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

In [5]:
OUTPUT_FOLDER = os.path.join(DATA_FOLDER, "output")
os.system(f"python3 preprocess_data.py --raw_data_path {DATA_FOLDER} --dest_path {OUTPUT_FOLDER}");

How many files were saved to `OUTPUT_FOLDER`?

In [6]:
len([name for name in os.listdir(OUTPUT_FOLDER) if os.path.isfile(os.path.join(OUTPUT_FOLDER, name))])

4

## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.


In [ ]:
os.system(f"python3 train.py --data_path {OUTPUT_FOLDER}");

Helper functions to run MLflow without blocking the execution of other cells

In [8]:
mlflow_process = None

def run_mlflow():
    global mlflow_process
    mlflow_process = subprocess.Popen(["python3", "-m", "mlflow", "ui"])

def stop_mlflow():
    global mlflow_process
    if mlflow_process:
        mlflow_process.terminate()
        mlflow_process = None

Launch MLflow UI

In [9]:
mlflow_thread = threading.Thread(target=run_mlflow)
mlflow_thread.setDaemon(True)
mlflow_thread.start()


What is the value of the `min_samples_split` parameter:

In [10]:
client = MlflowClient();

# Get experiment ID
experiment_name = "nyc-green-taxi-random-forest"
experiment = client.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id

print(f"Experiment ID: {experiment_id}")

# Get latest run
latest_run = client.search_runs([experiment_id], order_by=["start_time DESC"], max_results=1)[0]
print(f"Run ID: {latest_run.info.run_id}")

# Get min_sample_split parameter
print(f"min_sample_split: {latest_run.data.params['min_samples_split']}")

Experiment ID: 673718353336324652
Run ID: feabfb47b5a24c159aa53483c894b32f
min_sample_split: 2


In [11]:
stop_mlflow()

## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.


In [12]:
def run_mlflow():
    global mlflow_process
    mlflow_process = subprocess.Popen(
            ["python3", "-m", "mlflow", "ui",
             "--backend-store-uri", "sqlite:///mlflow.db",
             "--default-artifact-root", "./artifacts"]
    )

In [13]:
mlflow_thread = threading.Thread(target=run_mlflow)
mlflow_thread.setDaemon(True)
mlflow_thread.start()

Q: In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

A: `default-artifact-root` (specifically `--default-artifact-root ./artifacts`)

## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

In [ ]:
os.system(f"python3 hpo.py --data_path {OUTPUT_FOLDER}");

What's the best validation RMSE that you got?

In [15]:
client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

experiment_name = "random-forest-hyperopt"
experiment = client.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id

runs = client.search_runs(experiment_ids=[experiment_id], filter_string="")

rmse_values = [run.data.metrics['rmse'] for run in runs if 'rmse' in run.data.metrics]
min_rmse = min(rmse_values) if rmse_values else None

print(f"Minimum RMSE: {min_rmse}")

Minimum RMSE: 5.335419588556921


## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

In [ ]:
os.system(f"python3 register_model.py --data_path {OUTPUT_FOLDER}");

What is the test RMSE of the best model?

5.060

In [17]:
stop_mlflow()